In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import tqdm
import matplotlib.pyplot as plt
from torch.utils.tensorboard import SummaryWriter
from  torch.cuda.amp import autocast
import matplotlib.pyplot as plt

In [ ]:
train_dataloader = torch.utils.data.DataLoader(
    datasets.CIFAR10('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1302,), (0.3069,))])),
    batch_size=32, shuffle=True)

test_dataloader = torch.utils.data.DataLoader(
    datasets.CIFAR10('../data', train=False, 
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1302,), (0.3069,)) 
                    ])),
    batch_size=500, shuffle=False)

In [4]:
class SimpleConvNet(nn.Module):
    def __init__(self, in_channels= 3, num_classes= 10):
        super(SimpleConvNet, self).__init__()
        self.conv_block_1 = nn.Sequential(nn.Conv2d(in_channels,12,4),
                                    nn.MaxPool2d(kernel_size=2),
                                    nn.ReLU(inplace=True))
        self.conv_block_2 = nn.Sequential(nn.Conv2d(12,48,4),
                                    nn.MaxPool2d(kernel_size=2),
                                    nn.ReLU(inplace=True))
        self.linear_1 = nn.Sequential(nn.Linear(7680, 256),
                                    nn.ReLU(inplace= True))
        self.linear2 = nn.Sequential(nn.Linear(256, num_classes),
                                    nn.ReLU(inplace= True))
        self.Softmax = nn.Softmax(num_classes)
    def forward(self, x):
        print(x.shape)
        x= self.conv_block_1(x)
        print(x.shape)
        x= self.conv_block_2(x)
        print(x.shape)
        x = torch.flatten(x, 1)
        print(x.shape)
        x= self.linear_1(x)
        x= self.linear_2(x)
        print(x.shape)
        x= self.Softmax(x)
        return x

In [2]:
def Train(loader, model, optimizer, loss_func, device):
    model.train()
    loop = tqdm(loader)
    for batch_idx, (X_train, y_train) in enumerate(loop):

        X_train, y_train = X_train.to(device), y_train.to(device)
        optimizer.zero_grad()
        #with torch.cuda.amp.autocast():
        predictions = model(X_train)
        loss = loss_func(predictions, y_train)

        loss.backward()
        optimizer.step()
        optimizer.step()

        loop.set_postfix(loss=loss.item())
    return loss.item()

In [ ]:
def test_fn(loader, model, loss_func, device):
    model.eval()
    test_score = 0
    with torch.no_grad():
        for data in loader:
            images, targets = data
            images = images.to(device)
            outputs = model(images)
            test_score += loss_func(outputs, targets)
    return test_score

In [ ]:
loss_fn = nn.CrossEntropyLoss()
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
#model = SimpleConvNet().to(device)
model = SimpleConvNet()
optimizer = torch.optim.SGD(params= model.parameters(), lr=0.001)
num_epochs = 20
writer = SummaryWriter('runs/Logger')
for epoch in tqdm(range(num_epochs), leave=True):
    train_loss = Train(loader = train_dataloader, model= model, optimizer= optimizer, loss_func = loss_fn, device= device)
    writer.add_scalar('Train_Loss', train_loss, epoch)
    val_cross_entropy = test_fn(loader= test_dataloader, model= model, device=device, loss_func = loss_fn)
    writer.add_scalar('Test_loss', val_cross_entropy, epoch)
    if epoch % 125 == 0:
      print("epoch", epoch,"   ",)
      print("Train Loss:", train_loss,"   ",)
      print("Test Loss:", val_cross_entropy)